In [1]:

import pandas as pd
import pickle
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from nltk.stem.porter import PorterStemmer
import re
import nltk
import numpy as np
from nltk.corpus import stopwords
import csv

In [2]:
youtube_df = pd.read_csv('product_caption.csv')
youtube_df = youtube_df.dropna()

In [3]:
youtube_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 138 entries, 0 to 149
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   video_id       138 non-null    object
 1   product_title  138 non-null    object
 2   company        138 non-null    object
 3   caption        138 non-null    object
dtypes: object(4)
memory usage: 5.4+ KB


In [4]:
text = youtube_df['caption']

In [5]:
all_stop_words = stopwords.words('english')
all_stop_words.remove('not')
ps = PorterStemmer()

def text_editer(text):
    """Takes the text and lowers the characters.. 
    It removes any symbols/punctuation. 
    It then applys the portStemmer to anything that is not a stopword and returns the review.."""
    review = text.lower()
    review = re.sub("<.+>|[^a-zA-Z]|z.+z", " ", review)
    review = review.split()
    review = [word for word in review if word not in set(all_stop_words)]
    review = " ".join(ps.stem(word) for word in review)
    return review

class EditingText(BaseEstimator, TransformerMixin):
    """Transforms the data using the text editor function and returns it as a numpy array."""
    def __init__(self):
        super().__init__()

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        data = np.array([text_editer(str(text)) for text in X]).flatten()
        return data

In [6]:
pickled_model = pickle.load(open('SGDmodel.pkl', 'rb'))

# Testing the Model with Youtube Data 

In [7]:
values = pickled_model.predict(text)

In [8]:
values

array([1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1])

In [9]:
youtube_df['predicted_value'] = values

In [10]:
youtube_df

,video_id,product_title,company,caption,predicted_value
0,kGyZn-kOgF8,Iphone 14 Pro Max,Apple,foreign[Music]14 pro Max[Music]I've been using...,1
1,Qf1nfq_FW5Q,Iphone 14 Pro Max,Apple,Hello everyone it's SpawnPoiint and I've been...,1
3,FztDT48cPgM,Iphone 14 Pro Max,Apple,apple did a killer job showing off theiphone 1...,1
4,IzbxX55UTIg,Iphone 14 Pro Max,Apple,hi everyone Aaron here for zollotech and the ...,1
5,AUsw5zTZPzc,Iphone 14 Pro Max,Apple,whoa the 14 plus is overheating like madApple ...,0
...,...,...,...,...,...
145,sUkyWEXwMd0,Google Pixel 6 Pro,Google,(upbeat music)- Hello everyone and welcome to ...,1
146,Cajlwzk_igw,Google Pixel 6 Pro,Google,So we're getting close to the Pixel 6 launch a...,1
147,YMlTSmusEmA,Google Pixel 6 Pro,Google,I just touched the new Google pixelphones we'v...,1
148,fKZWeUywTW4,Google Pixel 6 Pro,Google,Unusable.I'm switching.Everyone keeps saying t...,1


In [11]:
youtube_df.count()

video_id           138
product_title      138
company            138
caption            138
predicted_value    138
dtype: int64

In [12]:
new_you_df = pd.DataFrame(columns=['Product', 'Company', '# Pos_Reviews', '# Neg_Reviews','Total_reviews'])
new_you_df['Product'] = youtube_df['product_title'].unique()
new_you_df['Company'] = youtube_df['company'].unique()

In [13]:
for product in youtube_df['product_title'].unique():
    #print(youtube_df['predicted_value'].where((youtube_df['product_title']==product) and (youtube_df['predicted_value'] == 1)).value_counts())
    pos = len(youtube_df.loc[youtube_df['predicted_value'].eq(1) & youtube_df['product_title'].eq(product)])
    neg = len(youtube_df.loc[youtube_df['predicted_value'].eq(0) & youtube_df['product_title'].eq(product)])
    
    print(pos, neg)
    
    new_you_df.loc[new_you_df['Product'].eq(product),'# Pos_Reviews'] = pos
    new_you_df.loc[new_you_df['Product'].eq(product),'# Neg_Reviews'] = neg

43 4
37 7
44 3


In [15]:
new_you_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Product        3 non-null      object
 1   Company        3 non-null      object
 2   # Pos_Reviews  3 non-null      object
 3   # Neg_Reviews  3 non-null      object
 4   Total_reviews  3 non-null      object
 5   Total_Rating   3 non-null      object
dtypes: object(6)
memory usage: 272.0+ bytes


In [18]:
new_you_df['Total_reviews'] = new_you_df['# Pos_Reviews'] + new_you_df['# Neg_Reviews']
new_you_df['Total_Rating'] = (new_you_df['# Pos_Reviews'] / new_you_df['Total_reviews']) * 10

new_you_df = new_you_df.astype({'# Pos_Reviews': int, '# Neg_Reviews': int, 'Total_reviews': int, "Total_Rating" : float})

In [19]:
new_you_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Product        3 non-null      object 
 1   Company        3 non-null      object 
 2   # Pos_Reviews  3 non-null      int64  
 3   # Neg_Reviews  3 non-null      int64  
 4   Total_reviews  3 non-null      int64  
 5   Total_Rating   3 non-null      float64
dtypes: float64(1), int64(3), object(2)
memory usage: 272.0+ bytes


In [23]:
new_you_df['Total_Rating'] = new_you_df['Total_Rating'].round(1)

In [24]:
new_you_df

,Product,Company,# Pos_Reviews,# Neg_Reviews,Total_reviews,Total_Rating
0,Iphone 14 Pro Max,Apple,43,4,47,9.1
1,Samsung Galaxy S22,Samsung,37,7,44,8.4
2,Google Pixel 6 Pro,Google,44,3,47,9.4


To CSV file

In [25]:
new_you_df.to_csv('product_rating.csv')

# To SQL Database

In [26]:
import sqlite3

In [27]:
conn = sqlite3.connect('test_database')
c = conn.cursor()

In [28]:
c.execute('CREATE TABLE IF NOT EXISTS products (Product text, \
          Company text, positive_review_number number, \
          negative_review_number number, total_review number, total_rating number)')
conn.commit()

In [29]:
new_you_df.to_sql('products', if_exists='replace', con=conn, index=False)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [30]:
c.execute('''  
		SELECT * FROM products
          ''')

In [31]:
df = pd.DataFrame(c.fetchall())    
print (df)


                     0        1   2  3   4    5
0   Iphone 14 Pro Max     Apple  43  4  47  9.1
1  Samsung Galaxy S22   Samsung  37  7  44  8.4
2  Google Pixel 6 Pro    Google  44  3  47  9.4
